In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as st
import pingouin as pg
%matplotlib inline 

In [2]:
data = pd.read_csv('/home/atrides/Desktop/R/statistics_with_Python/08_LogisticRegression/Data_Files/condom.dat', sep='\t')

In [3]:
data.head()

,particip,safety,use,gender,sexexp,previous,selfcon,perceive
0,5,3,Unprotected,Female,5,No Condom,5,4
1,6,1,Unprotected,Male,3,No Condom,2,2
2,9,0,Unprotected,Female,2,No Condom,3,0
3,13,3,Unprotected,Male,3,No Condom,4,4
4,14,2,Unprotected,Female,3,No Condom,6,3


In [5]:
data['use'].unique()

array(['Unprotected', 'Condom Used'], dtype=object)

In [6]:
data['usex'] = data['use'].replace({'Unprotected':0, 'Condom Used':1 })
data['genderx'] = data['gender'].replace({'Male':0, 'Female':1})
data['previousx'] = data['previous'].replace({'No Condom':0, 'Condom used':1, 'First Time with partner':2})

In [8]:
dummies = pd.get_dummies(data['previous'])
dummy = dummies.iloc[:,0:2]
print(dummy.head())

   Condom used  First Time with partner
0            0                        0
1            0                        0
2            0                        0
3            0                        0
4            0                        0


In [9]:
data = pd.merge(data, dummy, how='inner', left_index=True, right_index=True)

In [10]:
data.head()

,particip,safety,use,gender,sexexp,previous,selfcon,perceive,usex,genderx,previousx,Condom used,First Time with partner
0,5,3,Unprotected,Female,5,No Condom,5,4,0,1,0,0,0
1,6,1,Unprotected,Male,3,No Condom,2,2,0,0,0,0,0
2,9,0,Unprotected,Female,2,No Condom,3,0,0,1,0,0,0
3,13,3,Unprotected,Male,3,No Condom,4,4,0,0,0,0,0
4,14,2,Unprotected,Female,3,No Condom,6,3,0,1,0,0,0


In [11]:
from statsmodels.tools.tools import add_constant

data = add_constant(data)

In [12]:
from statsmodels.genmod import families

In [13]:
m01 = sm.GLM(data['usex'] , data[['const','safety','perceive','genderx']], 
          family=families.Binomial()).fit(attach_wls=True, atol=1e-10)
print(m01.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   usex   No. Observations:                  100
Model:                            GLM   Df Residuals:                       96
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -52.885
Date:                Wed, 16 Sep 2020   Deviance:                       105.77
Time:                        01:40:47   Pearson chi2:                     101.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4761      0.752     -3.294      0.0

In [15]:
m02 = sm.GLM(data['usex'] , data[['const','safety','perceive','genderx','First Time with partner','Condom used', 'selfcon', 'sexexp']], 
          family=families.Binomial()).fit(attach_wls=True, atol=1e-10)
print(m02.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   usex   No. Observations:                  100
Model:                            GLM   Df Residuals:                       92
Model Family:                Binomial   Df Model:                            7
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -43.986
Date:                Wed, 16 Sep 2020   Deviance:                       87.971
Time:                        01:41:36   Pearson chi2:                     87.8
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [16]:
data['predict'] = m02.fittedvalues

In [17]:
data.iloc[[11,52,74]]

,const,particip,safety,use,gender,sexexp,previous,selfcon,perceive,usex,genderx,previousx,Condom used,First Time with partner,predict
11,1.0,43,3,Unprotected,Female,1,No Condom,1,6,0,1,0,0,0,0.454654
52,1.0,83,5,Unprotected,Male,3,Condom used,7,6,0,0,1,1,0,0.915625
74,1.0,56,2,Condom Used,Female,2,Condom used,4,3,1,1,1,1,0,0.441225


### A female who used a condom in her previous encounter with her new partner scores 2 on all variables except perceived risk (for which she scores 6). Use the model to estimate the probability that she will use a condom in her next encounter. 

In [18]:
m02.params

const                     -4.959739
safety                    -0.482460
perceive                   0.949088
genderx                    0.002656
First Time with partner   -0.016615
Condom used                1.087196
selfcon                    0.347626
sexexp                     0.180423
dtype: float64

In [22]:
Z = m02.params['const'] + 2*m02.params['safety'] + 6* m02.params['perceive'] + 1* m02.params['genderx']+m02.params['Condom used'] + 2*m02.params['selfcon'] + 2*m02.params['sexexp']

In [23]:
Z

1.9158214940637062

In [24]:
probablity = 1/(1+np.exp(-Z))
probablity

0.8716717510017506

### Therefore, there is a 87% chance that she will use a condom on her next encounter.